# Toscanini BYOC — OpenMM Equilibration
**Audit ID:** `REPLACE_WITH_YOUR_AUDIT_ID`  
**Verdict:** `VETO`  
**Violations:** `LAW-125, LAW-130`  
**Platform:** Google Colab (NVIDIA T4)  
**Simulation:** 2 ns equilibration

> ⚠️ **Before running:** Upload your PDB file to this session's files panel.  
> After the run completes, this notebook will automatically post the refined  
> structure back to Toscanini for re-certification.


## Step 1: Install Dependencies

In [ ]:
pip install -q openmm pdbfixer
print("Dependencies installed.")

## Step 2: Upload Your Structure

In [ ]:
# Upload your PDB file using the Files panel (📁) on the left.
# Then set the filename below:
PDB_FILE   = "your_structure.pdb.pdb"      # ← rename if needed
OUTPUT_PDB = "your_structure.pdb_refined.pdb"
print(f"Input:  {PDB_FILE}")
print(f"Output: {OUTPUT_PDB}")

## Step 3: Run OpenMM Equilibration

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import sys

TEMPERATURE_K = 300
TIMESTEP_FS   = 2
SIM_STEPS     = 1000000   # 2 ns
REPORT_EVERY  = 5000

print("Toscanini OpenMM Protocol")
print(f"Audit ID  : REPLACE_WITH_YOUR_AUDIT_ID")
print(f"Violations: LAW-125, LAW-130")
print(f"Simulation: 2 ns at 2 fs/step")

pdb        = PDBFile(PDB_FILE)
forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")
modeller   = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield, pH=7.0)
modeller.addSolvent(forcefield, model="tip3p", padding=10*angstroms)

system = forcefield.createSystem(
    modeller.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometers,
    constraints=HBonds,
)
integrator = LangevinMiddleIntegrator(
    TEMPERATURE_K * kelvin,
    1.0 / picosecond,
    TIMESTEP_FS * femtoseconds,
)

try:
    platform = Platform.getPlatformByName("CUDA")
    print("Using CUDA GPU")
except Exception:
    platform = Platform.getPlatformByName("CPU")
    print("Using CPU (no GPU detected)")

simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

print("Running energy minimization...")
simulation.minimizeEnergy(maxIterations=1000)

simulation.reporters.append(
    StateDataReporter(sys.stdout, REPORT_EVERY, step=True,
                      potentialEnergy=True, temperature=True,
                      progress=True, remainingTime=True,
                      totalSteps=SIM_STEPS)
)

print(f"Running {SIM_STEPS * TIMESTEP_FS / 1e6:.1f} ns equilibration...")
simulation.step(SIM_STEPS)

positions = simulation.context.getState(getPositions=True).getPositions()
with open(OUTPUT_PDB, "w") as f:
    PDBFile.writeFile(simulation.topology, positions, f)

print(f"Refined structure saved: {OUTPUT_PDB}")

## Step 4: Post Results to Toscanini

In [ ]:
import requests

CALLBACK_TOKEN = "PASTE_YOUR_CALLBACK_TOKEN_HERE"
BACKEND_URL    = "http://localhost:8000"
CALLBACK_URL   = f"{BACKEND_URL}/refinement/callback"

print("Posting refined structure to Toscanini for re-certification...")

with open(OUTPUT_PDB, "rb") as f:
    response = requests.post(
        CALLBACK_URL,
        data={"token": CALLBACK_TOKEN},
        files={"file": (OUTPUT_PDB, f, "chemical/x-pdb")},
        timeout=120
    )

if response.status_code == 200:
    result = response.json()
    print("✅ Re-certification complete!")
    print(f"   Original audit : REPLACE_WITH_YOUR_AUDIT_ID")
    print(f"   Refined audit  : {result.get('refined_audit_id', 'N/A')}")
    print(f"   New verdict    : {result.get('verdict', 'N/A')}")
    print(f"   Coverage       : {result.get('coverage_pct', 'N/A')}%")
    print(f"   View comparison: {result.get('comparison_url', 'N/A')}")
else:
    print(f"⚠️ Callback failed ({response.status_code}): {response.text[:200]}")
    print("Manual option: Download the refined PDB and upload via the Toscanini dashboard.")


## Done
Your refined structure has been submitted for re-certification.  
Return to the Toscanini dashboard to view the before/after comparison.

> **Note:** The callback token is single-use and expires in 7 days.
